In [121]:
import pandas as pd
import numpy as np
import os
import re
import math

FutureList = ['ag', 'au', 'cu', 'IF','j', 'm', 'rb', 'ru']
FilePath = u"rank_price/"
FileName = 'rank_price.xlsx'
Columns = ['Future', 
           'PricePointNum', 'LabelNum', 'HiddenStateNum', 'RankWeight', 'PatternWeight', 
           'Sharpe', 'AnnualReturn', 'MaxDrawdown', 'WinRate']

In [122]:
def calculate(values):
    returns = []
    maxnow = values[0]
    maxdrawdown = 0
    tot = 0
    win = 0
    for j in range(1, len(values)):
        maxnow = max(maxnow, values[j])
        maxdrawdown = max(maxdrawdown, (maxnow - values[j]) / maxnow)
        if not (values[j] - values[j - 1]) == 0:
            tot += 1
        if values[j] - values[j - 1] > 0:
            win += 1
        returns.append((values[j] - values[j - 1]) / values[j - 1] - 0.04 / 252)    
    #计算年化夏普，无风险年化收益率取0.04，对标十年期国债利率，每年交易日认为是252个
    sharpe = np.sqrt(252) * np.mean(returns) / np.std(returns)
    annualreturn = math.sqrt(values[-1] / values[0]) - 1
    return sharpe, annualreturn, maxdrawdown, float(win) / float(tot)

In [123]:
def result(fut, path):
    ret = pd.DataFrame(columns = Columns)
    for rec in os.walk(path):
        path_now = rec[0]
        filelist = rec[2]
        for filename in filelist:
            path_name = path_now + '/' + filename
            data = pd.read_csv(path_name, header = 0, index_col = 0).T
            #print data
            #print data.columns
            if fut in data.columns:
                sharpe, annual_return, max_drawdown, win_rate = calculate(data[fut])
                #print [sharpe, annual_return, max_drawdown, win_rate]
                sl = filename.split('_')
                price_point_num = int(re.findall(r'-?\d+\.?\d*e?-?\d*?', sl[2])[0])
                label_num = int(re.findall(r'-?\d+\.?\d*e?-?\d*?', sl[3])[0])
                hidden_state_num = int(re.findall(r'-?\d+\.?\d*e?-?\d*?', sl[4])[0])
                rank_weight = float(re.findall(r'-?\d+\.?\d*e?-?\d*?', sl[6])[0])
                pattern_weight = float(re.findall(r'-?\d+\.?\d*e?-?\d*?', sl[7])[0])
                
                line = pd.DataFrame(columns = Columns)
                line['Future'] = (pd.Series([fut]))
                line['PricePointNum'] = (pd.Series([price_point_num]))
                line['LabelNum'] = (pd.Series([label_num]))
                line['HiddenStateNum'] = (pd.Series([hidden_state_num]))
                line['RankWeight'] = (pd.Series([rank_weight]))
                line['PatternWeight'] = (pd.Series([pattern_weight]))
                line['Sharpe'] = (pd.Series([sharpe]))
                line['AnnualReturn'] = (pd.Series([annual_return]))
                line['MaxDrawdown'] = (pd.Series([max_drawdown]))
                line['WinRate'] = (pd.Series([win_rate]))
                ret = ret.append(line, ignore_index=True)
                
    return ret

In [124]:
writer = pd.ExcelWriter(FileName)
All = pd.DataFrame(columns = Columns)
for future in FutureList:
    res = result(future, FilePath)
    res.to_excel(writer, future, index = False)
    All = All.append(res)
All.to_excel(writer, 'All', index = False)
writer.save()